In [33]:
def pos2str(pos):
    return f"{pos[0]}-{pos[1]}"
    
class elf:
    def __init__(self, position):
        self.position = position
        self.target = position
        self.moveDir = -1
        
    def pos2str(self, pos):
        return f"{pos[0]}-{pos[1]}"
    
    def resetTarget(self):
        self.target = self.position
        
    def moveToTarget(self):
        self.position = self.target
        
    def tryNorth(self, elfPositions):
        if (self.pos2str([self.position[0],self.position[1] - 1]) not in elfPositions 
                and self.pos2str([self.position[0] - 1,self.position[1] - 1]) not in elfPositions 
                and self.pos2str([self.position[0] + 1,self.position[1] - 1]) not in elfPositions):
            newPos = [self.position[0],self.position[1]-1]
            self.target = newPos
            return self.pos2str(newPos)
        return self.pos2str(self.position)
        
    def trySouth(self, elfPositions):
        if (self.pos2str([self.position[0],self.position[1] + 1]) not in elfPositions 
                and self.pos2str([self.position[0] - 1,self.position[1] + 1]) not in elfPositions 
                and self.pos2str([self.position[0] + 1,self.position[1] + 1]) not in elfPositions):
            newPos = [self.position[0],self.position[1]+1]
            self.target = newPos
            return self.pos2str(newPos)
        return self.pos2str(self.position)
        
    def tryWest(self, elfPositions):
        if (self.pos2str([self.position[0] - 1,self.position[1]]) not in elfPositions 
                and self.pos2str([self.position[0] - 1,self.position[1] + 1]) not in elfPositions 
                and self.pos2str([self.position[0] - 1,self.position[1] - 1]) not in elfPositions):
            newPos = [self.position[0]-1,self.position[1]]
            self.target = newPos
            return self.pos2str(newPos)
        return self.pos2str(self.position)
        
    def tryEast(self, elfPositions):
        if (self.pos2str([self.position[0] + 1,self.position[1]]) not in elfPositions 
                and self.pos2str([self.position[0] + 1,self.position[1] + 1]) not in elfPositions 
                and self.pos2str([self.position[0] + 1,self.position[1] - 1]) not in elfPositions):
            newPos = [self.position[0]+1,self.position[1]]
            self.target = newPos
            return self.pos2str(newPos)
        return self.pos2str(self.position) 
    
    def tryNeighbors(self, elfPositions):
        if (self.pos2str([self.position[0] + 1,self.position[1]]) not in elfPositions and
            self.pos2str([self.position[0] + 1,self.position[1] + 1]) not in elfPositions and
            self.pos2str([self.position[0] + 1,self.position[1] - 1]) not in elfPositions and
            self.pos2str([self.position[0] - 1,self.position[1]]) not in elfPositions and
            self.pos2str([self.position[0] - 1,self.position[1] + 1]) not in elfPositions and
            self.pos2str([self.position[0] - 1,self.position[1] - 1]) not in elfPositions and
            self.pos2str([self.position[0],self.position[1] + 1]) not in elfPositions and
            self.pos2str([self.position[0],self.position[1] - 1]) not in elfPositions):
            return True
        return False
        
    def considerMove(self, elfPositions):
        self.moveDir = (self.moveDir + 1) % 4
        # Check if moving is necessary
        if self.tryNeighbors(elfPositions):
            return self.pos2str(self.position)
        
        # North first
        # South second
        # West third
        # East last
        # in rotating order
        #print((self.moveDir))
        posStr = self.pos2str(self.position)
        for m in range(4):
            match((m + self.moveDir) % 4):
                case 0:
                    posStr = self.tryNorth(elfPositions)
                    if posStr != self.pos2str(self.position):
                        return posStr
                case 1:
                    posStr = self.trySouth(elfPositions)
                    if posStr != self.pos2str(self.position):
                        return posStr
                case 2:
                    posStr = self.tryWest(elfPositions)
                    if posStr != self.pos2str(self.position):
                        return posStr
                case 3:
                    posStr = self.tryEast(elfPositions)
                    if posStr != self.pos2str(self.position):
                        return posStr
        return posStr
        
def printGrid(elfPositions):
    minX = elfList[0].position[0]
    maxX = elfList[0].position[0]
    minY = elfList[0].position[1]
    maxY = elfList[0].position[1]
    for e in elfList:
        minX = min(e.position[0], minX)
        maxX = max(e.position[0], maxX)
        minY = min(e.position[1], minY)
        maxY = max(e.position[1], maxY)
    
    gridSize = (maxX - minX + 1) * (maxY - minY + 1)
    print(f"{maxX}-{minX} : {maxY}-{minY}")
    result = gridSize - len(elfList)
    print(result)
    
    gridOffsetX = 0 - minX
    gridOffsetY = 0 - minY
    print(f"{gridOffsetX}:{gridOffsetY}")
    #print(elfPositions)
    grid = [["." for x in range(maxX-minX+1)]for y in range(maxY-minY+1)]
    for e in elfList:
        grid[e.position[1] + gridOffsetY][e.position[0] + gridOffsetX] = "#"
        
    for y in grid:
        l = ""
        for x in y:
            l += x
        print(l)
    print()
            
elfPositions = {}
elfList = []

with open("input23.txt") as elfMap:
    for y, line in enumerate(elfMap):
        for x, char in enumerate(line):
            if char == "#":
                e = elf([x,y])
                elfList.append(e)
                elfPositions[pos2str([x,y])] = e
                
for n in range(rounds):
    #printGrid(elfPositions)
    # Iterate elves and propose moves
    targetPositions = {}
    for e in elfList:
        targetPos = e.considerMove(elfPositions)
        if targetPos in targetPositions:
            targetPositions[targetPos].resetTarget()
            e.resetTarget()
        else:
            targetPositions[targetPos] = e
    # Move elves
    for e in elfList:
        e.moveToTarget()
    # Rebuild currentPositions
    elfPositions.clear()
    for e in elfList:
        elfPositions[pos2str(e.position)] = e
        
## Get final result
#minX = elfList[0].position[0]
#maxX = elfList[0].position[0]
#minY = elfList[0].position[1]
#maxY = elfList[0].position[1]
#for e in elfList:
#    minX = min(e.position[0], minX)
#    maxX = max(e.position[0], maxX)
#    minY = min(e.position[1], minY)
#    maxY = max(e.position[1], maxY)
#    
#gridSize = (maxX - minX) * (maxY - minY)
#print(f"{maxX}-{minX} : {maxY}-{minY}")
#result = gridSize - len(elfList)
#print(result)
#
#grid = [["." for x in range(maxX-minX)]for y in range(maxY-minY)]
#for e in elfList:
#    grid[e.position[1]][e.position[0]] = "#"
#    
#for y in grid:
#    l = ""
#    for x in y:
#        l += x
#    print(l)
printGrid(elfPositions)

74--4 : 75--4
3762
4:4
................#..............................................................
.....#..#.#..#...#....#.#..#....#................#......#..#.#..#...#..#.......
..........................#..#...#.....#..#...##....#.........#.........#......
......#..#..#....###.............#................#.#.#.#.#.......#..#.........
.......#......#..#....#.#..#..#....###..##..##..#...........#..##..#.#..#.#.#..
...#.....##..#.#..#.#.....#..#..#.#...#.##......#.##.#.#...#.#.#...##..........
......#..##.#.#.#..#.#####..#.##.#.#.#....#.#.##.#..#.#...#.#.#..#.#.##........
#...##..#..#.#.#..#.#.....####..#.#.#..#.#.#.#..#.#...#.##.#.#..#.#.#...##.##..
......#..#.##...##.###.#.#.#..##...#.#..#.#.#.##.#.##.#...#.#..#.##.#.#..#.....
..##....#.#.#.#.#.#.###.#.#..#..#.#..#.#.#.#.#..###..##..#....#.#....#...#.#.#.
...#..#..#.#.#.#.#..#.##...###.#.#.#...#.##.#..#....#.#....#.#.#.##.#..#.......
..#..#.####.#.#.#.##.#.#..#...#.#.#.#.#.#....##.###..#..#.#.#.#.#..#.#.##...#..
..#..#.#####.#.#.

In [35]:
def pos2str(pos):
    return f"{pos[0]}-{pos[1]}"
    
class elf:
    def __init__(self, position):
        self.position = position
        self.target = position
        self.moveDir = -1
        
    def pos2str(self, pos):
        return f"{pos[0]}-{pos[1]}"
    
    def resetTarget(self):
        self.target = self.position
        
    def moveToTarget(self):
        self.position = self.target
        
    def tryNorth(self, elfPositions):
        if (self.pos2str([self.position[0],self.position[1] - 1]) not in elfPositions 
                and self.pos2str([self.position[0] - 1,self.position[1] - 1]) not in elfPositions 
                and self.pos2str([self.position[0] + 1,self.position[1] - 1]) not in elfPositions):
            newPos = [self.position[0],self.position[1]-1]
            self.target = newPos
            return self.pos2str(newPos)
        return self.pos2str(self.position)
        
    def trySouth(self, elfPositions):
        if (self.pos2str([self.position[0],self.position[1] + 1]) not in elfPositions 
                and self.pos2str([self.position[0] - 1,self.position[1] + 1]) not in elfPositions 
                and self.pos2str([self.position[0] + 1,self.position[1] + 1]) not in elfPositions):
            newPos = [self.position[0],self.position[1]+1]
            self.target = newPos
            return self.pos2str(newPos)
        return self.pos2str(self.position)
        
    def tryWest(self, elfPositions):
        if (self.pos2str([self.position[0] - 1,self.position[1]]) not in elfPositions 
                and self.pos2str([self.position[0] - 1,self.position[1] + 1]) not in elfPositions 
                and self.pos2str([self.position[0] - 1,self.position[1] - 1]) not in elfPositions):
            newPos = [self.position[0]-1,self.position[1]]
            self.target = newPos
            return self.pos2str(newPos)
        return self.pos2str(self.position)
        
    def tryEast(self, elfPositions):
        if (self.pos2str([self.position[0] + 1,self.position[1]]) not in elfPositions 
                and self.pos2str([self.position[0] + 1,self.position[1] + 1]) not in elfPositions 
                and self.pos2str([self.position[0] + 1,self.position[1] - 1]) not in elfPositions):
            newPos = [self.position[0]+1,self.position[1]]
            self.target = newPos
            return self.pos2str(newPos)
        return self.pos2str(self.position) 
    
    def tryNeighbors(self, elfPositions):
        if (self.pos2str([self.position[0] + 1,self.position[1]]) not in elfPositions and
            self.pos2str([self.position[0] + 1,self.position[1] + 1]) not in elfPositions and
            self.pos2str([self.position[0] + 1,self.position[1] - 1]) not in elfPositions and
            self.pos2str([self.position[0] - 1,self.position[1]]) not in elfPositions and
            self.pos2str([self.position[0] - 1,self.position[1] + 1]) not in elfPositions and
            self.pos2str([self.position[0] - 1,self.position[1] - 1]) not in elfPositions and
            self.pos2str([self.position[0],self.position[1] + 1]) not in elfPositions and
            self.pos2str([self.position[0],self.position[1] - 1]) not in elfPositions):
            return True
        return False
        
    def considerMove(self, elfPositions):
        self.moveDir = (self.moveDir + 1) % 4
        # Check if moving is necessary
        if self.tryNeighbors(elfPositions):
            return self.pos2str(self.position)
        
        # North first
        # South second
        # West third
        # East last
        # in rotating order
        #print((self.moveDir))
        posStr = self.pos2str(self.position)
        for m in range(4):
            match((m + self.moveDir) % 4):
                case 0:
                    posStr = self.tryNorth(elfPositions)
                    if posStr != self.pos2str(self.position):
                        return posStr
                case 1:
                    posStr = self.trySouth(elfPositions)
                    if posStr != self.pos2str(self.position):
                        return posStr
                case 2:
                    posStr = self.tryWest(elfPositions)
                    if posStr != self.pos2str(self.position):
                        return posStr
                case 3:
                    posStr = self.tryEast(elfPositions)
                    if posStr != self.pos2str(self.position):
                        return posStr
        return posStr
        
def printGrid(elfPositions):
    minX = elfList[0].position[0]
    maxX = elfList[0].position[0]
    minY = elfList[0].position[1]
    maxY = elfList[0].position[1]
    for e in elfList:
        minX = min(e.position[0], minX)
        maxX = max(e.position[0], maxX)
        minY = min(e.position[1], minY)
        maxY = max(e.position[1], maxY)
    
    gridSize = (maxX - minX + 1) * (maxY - minY + 1)
    print(f"{maxX}-{minX} : {maxY}-{minY}")
    result = gridSize - len(elfList)
    print(result)
    
    gridOffsetX = 0 - minX
    gridOffsetY = 0 - minY
    print(f"{gridOffsetX}:{gridOffsetY}")
    #print(elfPositions)
    grid = [["." for x in range(maxX-minX+1)]for y in range(maxY-minY+1)]
    for e in elfList:
        grid[e.position[1] + gridOffsetY][e.position[0] + gridOffsetX] = "#"
        
    for y in grid:
        l = ""
        for x in y:
            l += x
        print(l)
    print()
            
elfPositions = {}
elfList = []

with open("input23.txt") as elfMap:
    for y, line in enumerate(elfMap):
        for x, char in enumerate(line):
            if char == "#":
                e = elf([x,y])
                elfList.append(e)
                elfPositions[pos2str([x,y])] = e
                
movingElves = list(elfList)
roundCount = 0
while len(movingElves) > 0:
    roundCount += 1
    movingElves.clear()
    # Iterate elves and propose moves
    targetPositions = {}
    for e in elfList:
        targetPos = e.considerMove(elfPositions)
        if targetPos != pos2str(e.position):
            movingElves.append(e)
            
        if targetPos in targetPositions:
            targetPositions[targetPos].resetTarget()
            e.resetTarget()
        else:
            targetPositions[targetPos] = e
    # Move elves
    for e in elfList:
        e.moveToTarget()
    # Rebuild currentPositions
    elfPositions.clear()
    for e in elfList:
        elfPositions[pos2str(e.position)] = e
        
print(roundCount)
#printGrid(elfPositions)

997
